In [1]:
import chromadb
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import requests
import os
import openai
from bs4 import BeautifulSoup
client = chromadb.PersistentClient(path="chroma_db") # Create a new client, or load an existing one from the given path.
collection = client.get_collection(name="chroma_db") # Get a collection object from an existing collection, by name. Will raise an exception if it's not found.
#collection = client.create_collection(name="chroma_db") # Create a new collection with the given name. Will raise an exception if it already exists.

import numpy as np
model = SentenceTransformer('KBLab/sentence-bert-swedish-cased')


c:\Users\ehsaaas\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#collection = client.get_collection(name="test") # Get a collection object from an existing collection, by name. Will raise an exception if it's not found.
collection = client.get_or_create_collection(name="chroma_db") # Get a collection object from an existing collection, by name. If it doesn't exist, create it.
#client.delete_collection(name="chroma_db") # Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible

In [2]:
def meta_dict(num_pairs, value_str):
    """
    Creates a list of dictionaries, each with the same key-value pair.
    
    Parameters:
    - num_pairs: The number of dictionaries to create.
    - value_str: The string to use as the value in each key-value pair.
    
    Returns:
    A list of dictionaries.
    """
    # Creating a list of dictionaries with the specified number of pairs
    # Each dictionary has a key 'key' with the provided value_str
    dict_list = [{"url": value_str} for _ in range(num_pairs)]
    return dict_list



In [3]:
import uuid

def create_ids(num_elements):
    """
    Creates a list of unique strings using UUIDs, based on the specified number of elements.
    
    Parameters:
    - num_elements: The number of unique strings to generate.
    
    Returns:
    A list of unique strings, each represented as a UUID.
    """
    # Generating a list of unique strings using UUID4
    unique_strings_list = [str(uuid.uuid4()) for _ in range(num_elements)]
    return unique_strings_list



In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=650,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [5]:
with open('web_scraping/all_urls.txt', 'r') as f:
    all_urls = [line.rstrip('\n') for line in f]
print(all_urls)


['https://www.1177.se/sjukdomar--besvar/lungor-och-luftvagar/inflammation-och-infektion-ilungor-och-luftror/om-covid-19--coronavirus/om-vaccin-mot-covid-19/fragor-och-svar-om-vaccination-mot-covid-19/fragor-och-svar-om-vaccination-mot-covid-19/kan-jag-vaccinera-mig/jag-fick-covid-19-for-flera-manader-sen-men-har-fortfarande-besvar.-ska-jag-vaccinera-mig/', 'https://www.1177.se/sjukdomar--besvar/lungor-och-luftvagar/inflammation-och-infektion-ilungor-och-luftror/om-covid-19--coronavirus/om-vaccin-mot-covid-19/fragor-och-svar-om-vaccination-mot-covid-19/fragor-och-svar-om-vaccination-mot-covid-19/fragor-om-bokning2/hur-bokar-jag-tid-for-vaccination-mot-covid-19-om-jag-har-en-god-man/', 'https://www.1177.se/sjukdomar--besvar/lungor-och-luftvagar/inflammation-och-infektion-ilungor-och-luftror/om-covid-19--coronavirus/om-vaccin-mot-covid-19/fragor-och-svar-om-vaccination-mot-covid-19/fragor-och-svar-om-vaccination-mot-covid-19/om-vaccinen/vilka-typer-av-vaccin-mot-covid-19-anvands-i-sverige

In [9]:
import time

# Record the start time
start_time = time.time()

with open('web_scraping/all_urls.txt', 'r') as f:
    all_urls = [line.rstrip('\n') for line in f]
print(all_urls)


# Define the base path to your folder
folder_path = 'web_scraping/all_texts'

# Define the range of your documents
start_index = 1  # Starting index
end_index =  2305   # Assume we have 5 documents for this example

# Iterate over the range of indices
for i in range(start_index, end_index+1):
    i_url = i-1
    # Construct the filename for each document
    filename = f"1177-text_{i}.txt"
    file_path = os.path.join(folder_path, filename)
    
    # Open and read the file
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    texts = text_splitter.create_documents([content])
    texts = texts[1:len(texts)]
    emb_list = []
    text_list = []
    #embdedd the documents, and addd embeddings and documents to seperate lists
    for text in texts:
        text = str(text)
        emb = model.encode(text)
        emb_list.append(emb.tolist())
        text_list.append(text)
    #variables for the collection.add
    url = all_urls[i_url]
    ids = create_ids(len(emb_list))
    url_dict = meta_dict(len(emb_list),url)
# Add the embeddings to the collection
    collection.add(embeddings=emb_list,documents=text_list,metadatas=url_dict,ids=ids)
    print(f"Added {filename} to the collection")

end_time = time.time()

# Calculate the duration
duration = end_time - start_time

print(f"The loop took {duration} seconds to complete.")



['https://www.1177.se/sjukdomar--besvar/lungor-och-luftvagar/inflammation-och-infektion-ilungor-och-luftror/om-covid-19--coronavirus/om-vaccin-mot-covid-19/fragor-och-svar-om-vaccination-mot-covid-19/fragor-och-svar-om-vaccination-mot-covid-19/kan-jag-vaccinera-mig/jag-fick-covid-19-for-flera-manader-sen-men-har-fortfarande-besvar.-ska-jag-vaccinera-mig/', 'https://www.1177.se/sjukdomar--besvar/lungor-och-luftvagar/inflammation-och-infektion-ilungor-och-luftror/om-covid-19--coronavirus/om-vaccin-mot-covid-19/fragor-och-svar-om-vaccination-mot-covid-19/fragor-och-svar-om-vaccination-mot-covid-19/fragor-om-bokning2/hur-bokar-jag-tid-for-vaccination-mot-covid-19-om-jag-har-en-god-man/', 'https://www.1177.se/sjukdomar--besvar/lungor-och-luftvagar/inflammation-och-infektion-ilungor-och-luftror/om-covid-19--coronavirus/om-vaccin-mot-covid-19/fragor-och-svar-om-vaccination-mot-covid-19/fragor-och-svar-om-vaccination-mot-covid-19/om-vaccinen/vilka-typer-av-vaccin-mot-covid-19-anvands-i-sverige

In [14]:
fråga = "vad är symtomen vid en hjärattack"

query = model.encode(fråga)
query_formatting = query.tolist()

#query to chroma
answer = collection.query(
    query_embeddings=[query_formatting],
    n_results=5, include=['documents', 'metadatas']
)
print(answer['documents'][0][0])


page_content='Symtom\nDet här är några vanliga symtom vid en hjärtinfarkt:\nDu har mycket ont i bröstet hela tiden och värken kan stråla ut i armarna eller ryggen.\nDu har en obehaglig känsla eller ett tryck i bröstet som även kan stråla upp mot halsen, käkarna och skuldrorna.\nDu har svårt att andas.\nDu kallsvettas.\nDu känner dig rädd och har ångest.\nDu har hjärtklappning och yrsel.\nDu har besvär som liknar influensa och samtidigt ont i bröstet.\nVaga symtom kan också vara infarkt'


1

In [43]:

filepath = 'c:\\Users\\ehsaaas\\osrs_chatbot\\web_scraping\\scraped_texts\\1177-text_200.txt'


with open(filepath, 'r', encoding='utf-8') as file:
 # Read the contents of the file
    content = file.read()

texts = text_splitter.create_documents([content])
texts = texts[1:len(texts)]

emb_list = []
text_list = []
for text in texts:
    text = str(text)
    emb = model.encode(text)
    emb_list.append(emb.tolist())
    text_list.append(text)

url = 'https://www.1177.se/skane/sjukdomar--besvar/nasa-och-hals/halsont/'
ids = create_ids(len(emb_list))
url_dict = meta_dict(len(emb_list),url)

# Add the embeddings to the collection
collection.add(
    embeddings=emb_list,
    documents=text_list,
    metadatas=url_dict,
    ids=ids)
#ext_emb = model.encode(texts[0])




In [47]:
print(len(url_dict))

4


In [19]:
dict_list = [{"key": "value"} for _ in range(5)]
print(dict_list)


[{'key': 'value'}, {'key': 'value'}, {'key': 'value'}, {'key': 'value'}, {'key': 'value'}]


In [16]:
print(emb_list[3])

[ 1.54934257e-01 -2.67447121e-02 -2.82856151e-02  1.34447023e-01
 -5.78547604e-02  8.88795555e-02  9.46622491e-02  4.68741171e-02
 -8.76730308e-03 -6.15470868e-04  7.89034739e-02 -1.14321113e-01
  1.06177635e-01  5.68480790e-02  3.23341638e-02 -4.60314602e-02
  9.87229869e-02 -1.14060780e-02 -6.17603399e-02 -5.05112763e-03
 -8.25635642e-02  1.58814058e-01  2.89513543e-02  3.38378102e-02
  9.68464166e-02 -1.39521563e-03  6.41748533e-02 -6.81673214e-02
 -9.42455698e-03 -1.69557065e-01  9.06193480e-02 -4.11773361e-02
 -3.41366529e-02 -1.09644219e-01  4.85968621e-06 -7.19570294e-02
  8.85092095e-02  6.13812134e-02 -1.43132299e-01  7.01988116e-02
  7.28184031e-03 -3.26009504e-02 -8.04432705e-02 -9.73063894e-03
  4.65257093e-02  2.09207041e-03  9.38035473e-02  1.12038262e-01
 -4.62429114e-02  1.00603558e-01 -2.01825779e-02 -1.05367057e-01
 -8.99649486e-02 -1.24349788e-01  1.91810980e-01 -1.10969031e-02
 -1.53572224e-02 -8.25344101e-02  6.83611929e-02 -3.19398299e-04
 -2.92294454e-02  9.89344

In [13]:
texts = str(texts)
print(type(texts))


<class 'str'>
[


In [34]:
print(type(texts))

<class 'list'>


In [ ]:
collection.add(
    embeddings=[[1.2, 2.3, 4.5], [6.7, 8.2, 9.2]],
    documents=["This is a document", "This is another document"],
    metadatas=[{"source": "my_source"}, {"source": "my_source"}],
    ids=["id1", "id2"]
)

In [3]:
#get the url of the documents
# url extraction list extraction from sitemat

url = "https://www.1177.se/00sitemap.xml"

response = requests.get(url)

sitemap_text = response.text

soup = BeautifulSoup(sitemap_text, 'lxml-xml')

all_urls = soup.find_all('loc')

#save 10 random urls for later_tests
import random
all_urls_list = []

for i in range(len(all_urls)):
  all_urls_list.append(all_urls[i].get_text())



In [4]:
print(len(all_urls_list))


2305


In [8]:
print(all_urls_list[-30:-1])

['https://www.1177.se/om-1177/nar-du-ringer-1177/', 'https://www.1177.se/fragor--svar/nationellt/', 'https://www.1177.se/fragor--svar/e-tjanster/', 'https://www.1177.se/hitta-vard/kontaktkort/', 'https://www.1177.se/om-1177/om-1177/', 'https://www.1177.se/test/idas-testkopia/', 'https://www.1177.se/test/ida-testar-opti-1/', 'https://www.1177.se/aktuellt/vaccination-mot-covid-19-och-influensa/', 'https://www.1177.se/om-1177/1177-appen/', 'https://www.1177.se/aktuellt/minska-risken-att-bli-sjuk-i-vinter/', 'https://www.1177.se/om-1177/1177-direkt/', 'https://www.1177.se/liv--halsa/sol-och-varme/', 'https://www.1177.se/kontakt/lamna-synpunkt/', 'https://www.1177.se/kontakt/kontakta-varden/', 'https://www.1177.se/kontakt/fragor-om-vad-som-galler-i-en-region/', 'https://www.1177.se/kontakt/kontaktformular-for-kommun/', 'https://www.1177.se/sok/', 'https://www.1177.se/hitta-vard/', 'https://www.1177.se/fragor--svar/', 'https://www.1177.se/liv--halsa/', 'https://www.1177.se/barn--gravid/', 'h

In [35]:
#scripts that takes in documents from folder and adds them to a list called documents
import os
#cfolder_path = 'c:\Users\ehsaaas\osrs_chatbot\web_scraping\scraped_texts'

# Set the path to your folder containing the text files
folder_path = 'web_scraping\scraped_texts'

# Initialize an empty list to hold the contents of each file
documents = []

# Loop through each file in the directory
for i in range(1,len(all_urls_list)+1):
#or filename in os.listdir(folder_path):
    # Check if the file is a text file
        # Construct the full file path
        filename = f'1177-text_{i}.txt'
        file_path = os.path.join(folder_path, filename)
        print(filename)
        # Open the file and read its contents
        with open(file_path, 'r', encoding='utf-8') as file:
            # Read the contents of the file
            content = file.read()
            # Append the contents to the list
            documents.append(content)

# Now file_contents list contains the contents of all text files in the folder



1177-text_1.txt
1177-text_2.txt
1177-text_3.txt
1177-text_4.txt
1177-text_5.txt
1177-text_6.txt
1177-text_7.txt
1177-text_8.txt
1177-text_9.txt
1177-text_10.txt
1177-text_11.txt
1177-text_12.txt
1177-text_13.txt
1177-text_14.txt
1177-text_15.txt
1177-text_16.txt
1177-text_17.txt
1177-text_18.txt
1177-text_19.txt
1177-text_20.txt
1177-text_21.txt
1177-text_22.txt
1177-text_23.txt
1177-text_24.txt
1177-text_25.txt
1177-text_26.txt
1177-text_27.txt
1177-text_28.txt
1177-text_29.txt
1177-text_30.txt
1177-text_31.txt
1177-text_32.txt
1177-text_33.txt
1177-text_34.txt
1177-text_35.txt
1177-text_36.txt
1177-text_37.txt
1177-text_38.txt
1177-text_39.txt
1177-text_40.txt
1177-text_41.txt
1177-text_42.txt
1177-text_43.txt
1177-text_44.txt
1177-text_45.txt
1177-text_46.txt
1177-text_47.txt
1177-text_48.txt
1177-text_49.txt
1177-text_50.txt
1177-text_51.txt
1177-text_52.txt
1177-text_53.txt
1177-text_54.txt
1177-text_55.txt
1177-text_56.txt
1177-text_57.txt
1177-text_58.txt
1177-text_59.txt
1177-t

In [8]:
print(all_urls_list[250])

https://www.1177.se/barn--gravid/vard-och-stod-for-barn/for-dig-som-ar-barn-eller-ung/film-sa-tvattar-du-handerna/


In [41]:
collection = client.create_collection(name="main_db")

In [3]:
# Step 1: Read the text file
file_path = 'c:\\Users\\ehsaaas\\osrs_chatbot\\web_scraping\\scraped_texts\\1177-text_200.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

embeddings = model.encode(text)
print(type(text))
print(embeddings)

<class 'str'>
[ 1.01068757e-01  1.82285234e-02 -4.58694994e-02 -2.59547234e-02
  8.17977916e-03  1.23663843e-01  4.36694026e-02  4.08256762e-02
  9.14020166e-02 -4.53024991e-02  1.36655703e-01 -1.40921846e-01
  1.11440159e-01  9.09777805e-02 -6.33351179e-03 -5.23743182e-02
 -2.76844036e-02  3.57995033e-02 -1.66169759e-02 -2.09581529e-04
 -6.66672513e-02  9.53603387e-02 -1.21942014e-01  5.49575724e-02
  7.50517920e-02  2.06291047e-03  1.23565249e-01  6.08695596e-02
  6.98938891e-02 -1.71570778e-01  6.98285326e-02 -6.27262518e-02
 -6.50465414e-02 -1.08033441e-01  4.91078754e-06 -2.67384667e-02
  7.61866495e-02  3.68275233e-02 -1.54583037e-01 -6.11244924e-02
 -1.39668854e-02 -2.09524944e-01 -2.59404704e-02 -4.17469954e-03
  5.87122552e-02 -3.57088782e-02  8.39195848e-02  1.94945097e-01
  6.03877716e-02  1.08858496e-01 -3.58417667e-02  3.71148102e-02
 -3.58375721e-02 -3.74493487e-02  1.85766742e-01 -2.33652610e-02
 -2.77407337e-02 -2.43268907e-02  1.23738647e-01 -7.14267194e-02
  6.8634390

In [43]:
# Step 3: Generate embeddings
documents_emb = model.encode(documents)

In [52]:
#documents_emb = documents_emb.tolist()
print(documents_emb[0])


[0.030523458495736122, 0.032325100153684616, 0.018614811822772026, -0.08329752832651138, 0.10397914052009583, 0.033671412616968155, -0.011655288748443127, -0.06903497129678726, 0.08629673719406128, 0.023332931101322174, 0.06487196683883667, 0.048563987016677856, 0.07280591130256653, -0.009429153986275196, -0.015147499740123749, -0.0841621458530426, 0.042435672134160995, 0.03959588333964348, 0.04356341436505318, 0.005943853873759508, -0.039527732878923416, 0.033891711384058, -0.10673335939645767, 0.05083644390106201, 0.0756506621837616, -0.027531931176781654, 0.041695091873407364, -0.00829345639795065, 0.08526808768510818, -0.19745157659053802, 0.1347225457429886, -0.10630714893341064, 0.052175700664520264, -0.10453653335571289, 4.7473977247136645e-06, -0.003310696454718709, 0.050734732300043106, -0.0031974122393876314, 0.030785053968429565, -0.13357870280742645, 0.06391604244709015, -0.1595466285943985, 0.05225343629717827, 0.04478386044502258, -0.020099734887480736, -0.090160250663757

In [54]:
#generate a list of 100 ids for the 100 documents, they should be strings

ids = [str(i) for i in range(2307)]

print(len(ids))

# Create a list of dictionaries, each with "url" as the key and the actual URL as the value
url_list_of_dicts = [{"url": url} for url in all_urls_list]


2307


In [91]:
fråga = "bäckenbottenbesvär"

query = model.encode(fråga)
query_formatting = query.tolist()

#query to chroma
answer = collection.query(
    query_embeddings=[query_formatting],
    n_results=1, include=['documents']
)
print(answer['documents'][0])


In [8]:
collection = client.get_collection(name="test_hassano") # Get a collection object from an existing collection, by name. Will raise an exception if it's not found.
#collection = client.get_or_create_collection(name="test") # Get a collection object from an existing collection, by name. If it doesn't exist, create it.
#client.delete_collection(name="my_collection") # Delete a collection and all associated embeddings, documents, and metadata. 

ValueError: Collection test_hassano does not exist.

In [55]:
def clean_docs(documents):
    doc_size = len(answer['documents'][0])
    cleaned = [documents[0][i].replace("page_content=", "") for i in range(doc_size)]
    cleaned = [cleaned[i].replace("\\n", " ") for i in range(doc_size)]
    cleaned = ' '.join(cleaned)

    return cleaned

docs = answer['documents']
print(answer['documents'])
print(clean_docs(docs))

[["page_content='Mer på 1177.se\\nSå äter du hälsosamt\\nHälsosamma matvanor kan bidra till att minska risken för att du ska få hjärt-kärlsjukdomar, övervikt, typ 2-diabetes och cancer. Du behöver inte förändra dina matvanor helt och hållet i ett enda steg. Kom ihåg att varje liten förändring kan göra stor skillnad.\\nBra att veta om näring\\nDu behöver få i dig olika näringsämnen varje dag för att kroppen ska må bra. Dessa näringsämnen är kolhydrater, fett, proteiner, vitaminer och mineraler.  Genom att äta varierat får du i dig det du behöver.\\nMin tonåring struntar i sin diabetes, vad ska jag göra?\\n1177\\n–tryggt om din hälsa och vård'", "page_content='Minska mängden rött kött och charkprodukter\\nSalta inte för mycket\\nBegränsa mängden socker\\nÄt lagom mycket\\nTallriksmodellen hjälper dig att hålla koll\\n1177\\n–tryggt om din hälsa och vård\\nPå 1177.se får du råd om hälsa och information om sjukdomar och vilka mottagningar du kan kontakta. Logga in för att läsa din journal 

In [72]:
fråga = "vad är skadorna av alkohol?"

query = model.encode(fråga)
query_formatting = query.tolist()

#query to chroma
answer = collection.query(
    query_embeddings=[query_formatting],
    n_results=6, include=['documents', 'metadatas']
)



# Take the retrieved document and use it as a prompt for the text generation model
prompt = f" svara på frågan nedan med hjälp av denna information: \n{clean_docs(answer['documents'])} \n \n frågan som ska svaras: {fråga}"

from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": "Du är en hjälpsam assistent, använd endast information jag tillhandahåller"},
    {"role": "user", "content": prompt},

  ]
)
print(completion.choices[0].message.content)
print(answer['metadatas'][0][0]['url'])
print(answer['metadatas'][0][1]['url'])
print(answer['metadatas'][0][2]['url'])

Skadorna av alkohol är flera och påverkar flera delar av kroppen. Här är några exempel:

1. Alkohol kan skada alla muskler i kroppen och påverka sjukdomar och besvär i muskler, leder och skelett, som benskörhet och gikt.
2. Alkohol kan påverka din fertilitet, göra det svårare för både män och kvinnor att bli gravid, och orsaka ovillig barnlöshet. Den kan också skada kvaliteten på spermier och leda till fosterskador och negativ påverka barnets utveckling.
3. Drickande av alkohol kan resultera i inflammation i bukspottkörteln och öka risken för diabetes.
4. Alkohol kan påverka huden genom att vidga blodkärlen, vilket kan göra ansiktet rödflammigt. Den kan även förvärra hudsjukdomar som eksem, psoriasis, akne och rosacea.
5. Alkohol kan orsaka skador på nerverna på flera sätt, bland annat genom brist på B-vitaminer och andra näringsämnen som kroppen behöver. Skadliga ämnen bildas när alkoholen bryts ner i kroppen.
6. Alkohol kan orsaka njursvikt, som gör att njurarna inte kan rena blodet 

In [70]:
print(prompt)

 svara på frågan nedan med hjälp av denna information: 
'Studier har visat att endometrios till viss del är ärftligt. Det innebär att sannolikheten är större att få endometrios om du har en genetisk släkting som har sjukdomen. Hos en del läker endometrios ut av sig själv. När du kommer iklimakterietoch slutar få mens brukar sjukdomen försvinna. Vad händer i kroppen? Endometrios kan börja som knappnålsstora knottror som efter hand växer till större områden som kallas endometrioshärdar. Du kan ha lindriga besvär även om du har stora endometrioshärdar. Hur ont det gör beror delvis på var förändringarna sitter.' 'Endometrios kan finnas i princip var som helst i kroppen Det är dock vanligast att endometrios finns på ett eller flera av följande ställen: bukhinnan i nedre delen av buken och vid äggstockarna ena eller båda äggstockarna utsidan av livmodern urinblåsan ändtarmen andra inre organ. Det är ganska vanligt att det bildas endometrioscystor i äggstockarna. De kallas ovarialendometriom 

In [62]:
print(prompt)

 källorna: ''Blodet kan vara mörkt eller svart och blandat i avföringen. När och var ska jag söka vård? Kontakta envårdcentralom du har blod i avföringen. Många mottagningar kan du kontakta genom attlogga in. Det finns sjukdomar som ger symtom i form av blod i avföringen. Kontakta därför alltid en vårdcentral om du har blod i avföringen. Om det är helg kan du vänta tills det blir vardag. Om det är bråttom Kontakta genast envårdcentraleller enjouröppen mottagningom du blöder kraftigt eller har ont. Om det är stängt, sök vård på en akutmottagning.' 'Om det är bråttom Kontakta genast envårdcentraleller enjouröppen mottagningom du blöder kraftigt eller har ont. Om det är stängt, sök vård på en akutmottagning. Ring telefonnummer 1177 om du vill hasjukvårdsrådgivning. Då får du hjälp att bedöma dina symtom eller hjälp med var du kan söka vård. Undersökningar och utredningar Det är viktigt att du blir undersökt av en läkare om du har blod i avföringen. Besvären går ofta över av sig själv och 